In [1]:
# Create an ImageJ gateway
import imagej
import functools

In [2]:
imagej_core_dep = 'net.imagej:imagej:2.1.0'  #'sc.fiji:fiji:2.1.1'
#ij = imagej.init(imagej_core_dep, headless=False)

myImageJPath = 'C:/Users/nicol/fiji-win64/Fiji.app'
ij = imagej.init(myImageJPath, headless=False)

In [3]:
# Import scijava classes using jimport
from scyjava import jimport
Command = jimport('org.scijava.command.Command')
JPlugin = jimport('org.scijava.plugin.Plugin')
Parameter = jimport('org.scijava.plugin.Parameter')
ImagePlus = jimport('ij.ImagePlus')

In [4]:
#jpype imports
import jpype
from jpype import JImplements, JOverride

In [5]:
# Pull in types
from jpype.types import *

In [6]:
# Creates a simple command
@JImplements(Command)
class SimpleCommand(object):
    @JOverride
    def run(self):
        print("Hello!")

In [7]:
# Instantiate it and run it
myCommand = SimpleCommand()
myCommand.run()

Hello!


In [8]:
# Problem:
# Is there a way to run this command using the command service ?
# Can this command appear in the ImageJ GUI ?
# Can we make parameters for this command which are recognized in Java?

In [9]:
# Different class which can be used as input or output parameter
print(JInt.class_)
print(ImagePlus.class_)
print(JString.class_)
print(type(JInt))
print(type(JInt.class_))

int
class ij.ImagePlus
class java.lang.String
<class '_jpype._JClass'>
<java class 'java.lang.Class'>


In [10]:
# Attempt at making a demo scijava plugin in python

class ScijavaGreetingsCommand:
    
    def set_name(self,name: JString):
        self.name = name
    
    def __init__(self):
        #No extra args accepted. 
        print("ScijavaGreetingsCommand object created")
        
    def run(self):
        print("Command is being run")
        print("Hello "+self.name)
    

In [11]:
#My wish:

def CommandInput(number):
    print("Useless called")
    def identity(func):
        #@functools.wraps(func)
        #def wrapper(*args, **kwargs):
        #    return func
        return func
    return identity

def repeat(num_times):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat
    return decorator_repeat

def useless(num_times):
    print("Useless called")
    def identity(func):
        #@functools.wraps(func)
        #def wrapper(*args, **kwargs):
        #    return func
        return func
    return identity

@CommandInput(3)
class MyPyCommand:        
    def run(self):
        print(self.name)
        print(self.familiar)
        greeting = JString("MyPyCommand is run")
        print(greeting)

@useless(8)
@repeat(2)        
class AnotherGreetingCommand:
    def __init__(self): # no param constructor compulsory TODO : can it be omitted ?
        print("AnotherGreetingCommand object created")

    @property
    def name(self):
        return self._name
    
    @name.setter # The decorator is used for discovery
    def name(self, name: JString):
        self._name = name # The code in this function will actually not be executed on the java side but it can be useful to keep it functional on the python side
    
    @property
    def greeting(self):
        return self._greeting
    
    def run(self):
        self._greeting = "Hello " + self.name

print(ij.context)

Useless called
Useless called
org.scijava.AbstractContextual.context


In [99]:

def ScijavaCommand(**kwargs):      
    print("- Registering scijava command")
    def registerCommand(func): 
        
        # This class will be registered as a SciJava Command
        builder = PyCommandBuilder() # Java PyCommandBuilder
        
        # The name of the command - to avoid name conflicts, consider a 'virtual' class name with its package
        builder = builder.name(kwargs['name'])
        
        # Register all inputs
        print('- Inputs')
        for name, javaClass in kwargs['inputs'].items():
            print('\t', name,' : ', javaClass)
            builder = builder.input(name, javaClass)
            setattr(func, name, None) # declares empty input field 
        print('Inputs registered')
        
        # Register all outputs
        print('- Outputs')
        for name, javaClass in kwargs['outputs'].items():
            print('\t', name,' : ', javaClass)
            builder = builder.output(name, javaClass)
            setattr(func, name, None) # declares empty output field
        print('Outputs registered')
                
        # Wraps the run function - takes kwargs as input, returns outputs
        def wrapped_run(inner_kwargs):
            print('Wrapped run called')
            print(inner_kwargs)
            for name, javaClass in kwargs['inputs'].items():
                print(name)
                print(str(inner_kwargs[name]))
                setattr(func, name, inner_kwargs[name]) # sets inputs 
            
            print('Inputs set!')
            print('Running function...')
            func.run(func)
            print('Run function done!')
            outputs = {}
            for name, javaClass in kwargs['outputs'].items():
                outputs[name] = getattr(func, name) # gets outputs
            return JObject(outputs, JClass('java.util.Map'))
        
        builder = builder.function(wrapped_run)
        
        # Effectively registers this command
        builder.create(kwargs['context']) 
        return func
    
    return registerCommand

@ScijavaCommand(context = ij.context(),
                name = 'pyCommand.MyPyCommand110',
                inputs = {'name': JString, 'familiar': JString},
                outputs = {'greetings': JString})
class MyPyCommand:        
        
    def run(self):
        self.greetings = 'Hello' + str(self.name)


- Registering scijava command
- Inputs
	 name  :  <java class 'JString'>
	 familiar  :  <java class 'JBoolean'>
Inputs registered
- Outputs
	 greetings  :  <java class 'JString'>
Outputs registered


[java.lang.Enum.toString] [ERROR] Module threw exception
java.lang.IllegalStateException: java.lang.InstantiationException: boolean
	at org.scijava.command.PyCommandInfo$PyCommandModuleItem.getDefaultValue(PyCommandInfo.java:354)
	at org.scijava.module.DefaultModuleService.save(DefaultModuleService.java:290)
	at org.scijava.module.DefaultModuleService.saveInput(DefaultModuleService.java:536)
	at org.scijava.module.DefaultModuleService.lambda$saveInputs$0(DefaultModuleService.java:341)
	at java.util.ArrayList.forEach(ArrayList.java:1259)
	at java.util.Collections$UnmodifiableCollection.forEach(Collections.java:1082)
	at org.scijava.module.DefaultModuleService.saveInputs(DefaultModuleService.java:341)
	at org.scijava.module.process.SaveInputsPreprocessor.process(SaveInputsPreprocessor.java:60)
	at org.scijava.module.ModuleRunner.preProcess(ModuleRunner.java:102)
	at org.scijava.module.ModuleRunner.run(ModuleRunner.java:154)
	at org.scijava.module.ModuleRunner.call(ModuleRunner.java:124)


In [94]:
mpc = MyPyCommand()
print(mpc.trux)

print(MyPyCommand.name)

AttributeError: 'MyPyCommand' object has no attribute 'trux'

In [13]:
#org.scijava.command.PyCommandBuilder

PyCommandBuilder = jimport('org.scijava.command.PyCommandBuilder')


In [52]:
@JImplements(Function)
class TestJavaFunction:
    @JOverride
    def apply(self, arg):
        print(self)
        print(str(arg['name']))
        print(arg['repeat'])
        print('Function executed with arg ?')
        print('Coucou coucou coucou')
        thegreeting = 'Hello '+str(arg['name'])
        print(thegreeting)
        return JObject({'greetings':thegreeting} , JClass('java.util.Map'))

myFunction = TestJavaFunction()
    
builder = PyCommandBuilder()
builder = builder.name("pyimagej.MyCommand18")
builder = builder.input("name", JString)
builder = builder.input("repeat", JString)
builder = builder.output("greetings", JString)
buider = builder.function(myFunction)
builder.create(ij.getContext())

       #.input("repeat", Integer.class)
       #.input("image", ImagePlus.class)
       #.output("greetings", String.class)
       #.menuPath("Plugins>DemoPyCommand")
       #.function(inputs -> {
       #             Map<String, Object> out = new HashMap<>();
       #             for (int i = 0;i<(int)inputs.get("repeat");i++) {
       #                 System.out.println("Repeat!");
       #             }
       #             String greetings = "Hello "+inputs.get("name")+"!";
       #             ((ImagePlus)inputs.get("image")).close();
       #             out.put("greetings", greetings);
       #             return out;
       #         })
       #         .create(ij.context());

Nico Nico Rom Oli 
3
Function executed with arg ?
Coucou coucou coucou
Hello Nico Nico Rom Oli 
Nico Nico Rom Oli 
3
Function executed with arg ?
Coucou coucou coucou
Hello Nico Nico Rom Oli 
Nico Nico Rom Oli 
3
Function executed with arg ?
Coucou coucou coucou
Hello Nico Nico Rom Oli 
I can't believe this work
3
Function executed with arg ?
Coucou coucou coucou
Hello I can't believe this work


In [28]:
Runnable = jimport('java.lang.Runnable')
Function = jimport('java.util.function.Function')
@JImplements(Runnable)
class TestJavaRunnable:
    @JOverride
    def run(self):
        print('Runnable executed')
        
myRunnable = TestJavaRunnable()
myRunnable.run()

@JImplements(Function)
class TestJavaFunction:
    @JOverride
    def apply(self, arg):
        print('Function executed with arg'+arg)
        return {'name': 'out', 'stuff': 'some stuff'}
        
myRunnable = TestJavaRunnable()
myRunnable.run()

myFunction = TestJavaFunction()
output = myFunction.apply('coucou')

for k, v in output.items():
    print('\t', k,' : ', v)

Runnable executed
Runnable executed
Function executed with argcoucou
	 name  :  out
	 stuff  :  some stuff


In [137]:
pyCommand1 = MyPyCommand()
pyCommand1.printTest(5)
print(pyCommand1.number)
pyCommand1.extrafunc = lambda x: x
pyCommand1.name = 'Nico'
pyCommand1.run()

pyCommand2 = MyPyCommand()
pyCommand2.printTest(3)
pyCommand2.name = 'Bob'
pyCommand2.run()
pyCommand1.run()


I'm a MyPy Command
5
Nico
Nico
Nico
Nico
Nico
MyPyCommand is run
I'm a MyPy Command
Bob
Bob
Bob
MyPyCommand is run
Nico
Nico
Nico
Nico
Nico
MyPyCommand is run


In [60]:
command = ScijavaGreetingsCommand()
command.set_name("XD")
command.run()


anotherc = AnotherGreetingCommand()
anotherc.name = "Nico"
anotherc.run()
print(anotherc.greeting)
print(anotherc.greeting)

ScijavaGreetingsCommand object created
Command is being run
Hello XD
AnotherGreetingCommand object created
AnotherGreetingCommand object created
Hello Nico
Hello Nico


In [51]:
# Access python class properties through reflection
# In order to make it match a scijava command

class ScijavaInput:
    def __init__(self, name, javaClass):
        self.name = name
        self.javaClass = javaClass # To be validated

class PythonCommandInvestigator:
    
    inputs = []
    outputs = []
    
    def __init__(self, python_class):
        self.python_class = python_class

    def listFields(self):
        print(dir(self.python_class))
        
    def parseInputs(self):
        for attr in dir(self.python_class):
            print(attr)
            

investigator = PythonCommandInvestigator(ScijavaGreetingsCommand)

print(investigator)

investigator.listFields()
investigator.parseInputs()

type(investigator)


['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'run', 'set_name']
__class__
__delattr__
__dict__
__dir__
__doc__
__eq__
__format__
__ge__
__getattribute__
__gt__
__hash__
__init__
__init_subclass__
__le__
__lt__
__module__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__sizeof__
__str__
__subclasshook__
__weakref__
run
set_name


__main__.PythonCommandInvestigator

In [40]:
l = [3,4,5]
print(l)
l.append(7)
print(l)
func = 'print'
a = 'coucou'
print(a)
eval(func+'("coucou with eval")')

command = ScijavaGreetingsCommand()
command.set_name('Bob')
command.run()
runMethodOfCommand = getattr(command,'run')
runMethodOfCommand
runMethodOfCommand = getattr(command,'run') # method bound to an instance of the object
runMethodOfCommand()

runMethodOfGreetings = getattr(ScijavaGreetingsCommand,'run') # class method
runMethodOfGreetings(command) # this needs a self an input because it's a class method


[3, 4, 5]
[3, 4, 5, 7]
coucou
coucou with eval
ScijavaGreetingsCommand object created
Command is being run
Hello Bob
Command is being run
Hello Bob
Command is being run
Hello Bob


In [20]:


ij.ui().showUI()

[java.lang.Enum.toString] [ERROR] Module threw exception
java.lang.NullPointerException
	at org.scijava.command.PyCommandInfo$1.run(PyCommandInfo.java:65)
	at org.scijava.module.ModuleRunner.run(ModuleRunner.java:165)
	at org.scijava.module.ModuleRunner.call(ModuleRunner.java:124)
	at org.scijava.module.ModuleRunner.call(ModuleRunner.java:63)
	at org.scijava.thread.DefaultThreadService.lambda$wrap$2(DefaultThreadService.java:225)
	at java.util.concurrent.FutureTask.run(FutureTask.java:266)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
[java.lang.Enum.toString] [ERROR] Module threw exception
java.lang.NullPointerException
	at org.scijava.command.PyCommandInfo$1.run(PyCommandInfo.java:65)
	at org.scijava.module.ModuleRunner.run(ModuleRunner.java:165)
	at org.scijava.module.ModuleRunner.call(ModuleRunner.java:124)
	at org.scija

In [14]:
CommandInfo = jimport('org.scijava.command.CommandInfo')

In [19]:


simpleCommandInfo = CommandInfo(JClass(SimpleCommand))

TypeError: JClass requires str or java.lang.Class instance, not 'type'

In [55]:
outputs = {}


In [56]:
outputs['ok'] = 'truc'

In [57]:
print(outputs)

{'ok': 'truc'}
